# [3.4] LM Agent Evaluations

# Setup (don't read just run)

In [15]:
import json
import os
os.chdir("c:\\Users\\styme\\OneDrive\\Documents\\AI STUFF\\Model Written Evals\\Code Replication\\ARENA_evals\\curriculum")
import wikipedia
from wikipedia import WikipediaPage
from wikipedia import DisambiguationError, PageError
from openai import OpenAI
from openai.types.chat.chat_completion_message_tool_call import ChatCompletionMessageToolCall
from anthropic import Anthropic
from utils import establish_client_OpenAI
from utils import retry_with_exponential_backoff
from pprint import pprint
from inspect_ai.model import ChatMessageUser, ChatMessageAssistant, ChatMessageSystem
import re
from utils import countrylist
from utils import evaluate_expression

# Test the function



# 1️⃣ Intro to LM Agents

## Resources:

- [OpenAI Function Calling Guide](https://platform.openai.com/docs/guides/function-calling)

- [Anthropic Function Calling Guide](https://docs.anthropic.com/en/docs/build-with-claude/tool-use)

- [Evaluating Language-Model Agents on Realistic Autonomous Tasks](https://evals.alignment.org/Evaluating_LMAs_Realistic_Tasks.pdf) (Kinniment et al., ARC Evaluations Team (now METR), 2023)

- [Large Language Models can Strategically Deceive their Users when Put Under Pressure](https://arxiv.org/pdf/2311.07590) (Scheurer et al., Apollo Research, ICLR 2024)

- [LLM Powered Autonomous Agents](https://lilianweng.github.io/posts/2023-06-23-agent/) (Lilian Weng, OpenAI Safety Team, 2023)

- [AXRP Episode 34 - AI Evaluations with Beth Barnes](https://www.alignmentforum.org/posts/vACr4DExfeRMaCoo7/axrp-episode-34-ai-evaluations-with-beth-barnes) (Daniel Filan, 2024)

- [Reflexion: Language Agents with Verbal Reinforcement Learning](https://arxiv.org/pdf/2303.11366) (Shinn et al., 2023)

- [Answering Questions by Meta-Reasoning over Multiple Chains of Thought](https://arxiv.org/pdf/2304.13007) (Yoran et al., 2024)

- [Toolformer: Language Models Can Teach Themselves to Use Tools](https://arxiv.org/pdf/2302.04761) (Schick et al., META AI Research, 2023)


LM agents are important and we should evaluate them.

They might be able to do more things.

Lots of threat models go through agentic behaviour.

Bad actors might scaffold agents and do bad things.

An agent consists of 4 main things.

- A 'reasoner' or 'reasoning agent.' Some people also call this a 'world model.' For our purposes this will be a large language model.

- Tools which allow the agent to act in the 'world.'

- Memory so that the agent can recall prior actions. This can either be:

    - Short-term memory: for our purposes this will be the context window

    - Long-term memory: there are many cases where context-windows are too short, and we will need to give the agent high-level information about actions it took a long time ago. We analogise this to 'long-term memory'.

- Scaffolding: This is essentially any structure which we provide to the 'reasoning engine' in order to help it to reason better, such as:

    - Prompting frameworks.

    - The ability to model plans into the future (at a high-level).

    - Using subagents to take care of trivial tasks.

    - Subgoal decomposition.

EXCALIDRAW!

# 2️⃣ A Basic LM agent

First, we will start by building a simple LM 'agent'. The task we'll get it to solve is an arithmetic task. LLMs struggle with arithmetic, but we can drastically improve their performance by providing a simple calculation tool.

### Exercise - Build a simple arithmetic problem
```c
Difficulty: 🔴🔴🔴⚪⚪
Importance: 🔵🔵⚪⚪⚪

You should spend up to 20-25 minutes on this exercise.
```

Build a class for a "game" that takes in two numbers, and creates a task consisting of `len(operation_list)` many problems of the form "Calculate `num1 operation_list[i] num2`". Once one problem is solved, move to the next one.

We'll try the model with and without tools on this task, and see how significantly performance improves.

<details><summary>Aside: Handling calculations</summary><br> When we handle the calculations for the model, technically we could use Python's <code>eval()</code> function (this is what <a href = "https://github.com/anthropics/anthropic-cookbook/blob/main/tool_use/calculator_tool.ipynb">Anthropic did</a>(!)). However, this function evaluates an arbitrary Python expression, and so allows AI models to run arbitrary code. In the long-run, we're trying to do these evaluations on models which we suspect of being dangerous; so even though we could probably trust the current suite of language models offered by OpenAI and Anthropic, we should get into good habits of not running arbitrary code outputted by language models (except in very carefully set-up environments). To this end, we've implemented an <code>evaluate_expression</code> function for you to use instead.</details>


In [16]:
class arithmeticProblem:
    def __init__(self, num1, num2):
        self.num1 = num1
        self.num2 = num2
        self.operation_list=["+","-","*","/", "%", "//"]
        self.answer_dict = {str(num1) + self.operation_list[i] + str(num2): str(evaluate_expression(str(num1) + self.operation_list[i] + str(num2))) for i in range(len(self.operation_list))}
        self.solved_dict = {str(num1) + self.operation_list[i] + str(num2): False for i in range(len(self.operation_list))}
        self.count_task = 0
        
    def getCurrentTask(self):
        #Returns the current arithmetic task
        return str(self.num1) + " " + self.operation_list[self.count_task] + " " + str(self.num2)
    
    def checkSolved(self):
        #Checks if all tasks are solved
        return all(self.solved_dict.values())
    
    def checkAnswer(self, answer : str):
        #Checks if the answer is correct. 
        # 
        # If you're testing the model without access to tools, you'll need to include some "wiggle room," as its answer may not be exact.
        
        if abs(float(self.answer_dict[str(self.num1) + self.operation_list[self.count_task] + str(self.num2)]) - float(answer)) <= 0.00001:
            return True
        else:
            return False
    def update(self):
        #Changes task once the current task is solved
        self.solved_dict[str(self.num1) + self.operation_list[self.count_task] + str(self.num2)] = True
        self.count_task+=1
        if self.count_task>len(self.operation_list)-1:
            self.count_task = 0
        
    def calculate(self, expression : str):
        #Calculates the expression. Use the eval function from utils.py for this.
        
        print(expression)
        return str(evaluate_expression(expression))
        
    
x = arithmeticProblem(10,15)
print(x.answer_dict)

{'10+15': '25.0', '10-15': '-5.0', '10*15': '150.0', '10/15': '0.6666666666666666', '10%15': '10.0', '10//15': '0.0'}


### Exercise - Define and implement a tool or list of tools for this task
```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 10-15 minutes on this exercise.
```

When you do this exercise, make sure you refer back to [OpenAI's function calling guide](https://platform.openai.com/docs/guides/function-calling). Also, Anthropic has a good example of what good and bad tool calling looks like [here](https://docs.anthropic.com/en/docs/build-with-claude/tool-use#example-poor-tool-description) which you might want to refer to. The main takeaway, as always with LLMs, is to **provide explicit descriptions**.

In [17]:
tool_list = [
    {
        "type" : "function",
        "function" : {
            "name" : "calculate",
            "description" : "Calculates the result of an arithmetic expression. For example, you could provide an input in the form \"2+3\" and the function would return 5. Or you could provide an expression like \"10/3\" and the function would return 3.3333333333333335.",
            "parameters" : {
                "type" : "object",
                "properties" : {
                    "expression" : {
                        "type" : "string",
                        "description" : "The arithmetic expression that you want to be evaluated."
                    }
                },
                "required" : ["expression"],
                "additionalProperties" : False
            }
        }
    },
]

Here are two functions that will be useful when designing agents. They take content and tool_call objects and return an expression which can be fed directly into the `ChatHistory`. If you're unsure about any aspect of them, then you should refer back to OpenAI's API documentation.

In [18]:
def tool_call_message(tool_call : ChatCompletionMessageToolCall, content : str):
    return {
        "role" : "tool",
        "tool_call_id" : tool_call.id,
        "name" : tool_call.function.name,
        "content" : content
    }

def user_message(content : str):
    return {
        "role" : "user",
        "content" : content
    }

### Exercise - Implement an LM agent class using the arithmetic task above
```c
Difficulty: 🔴🔴🔴🔴⚪
Importance: 🔵🔵🔵🔵🔵

You should spend up to 20-25 minutes on this exercise.
```

Build out the following simple agent class.

In [19]:
class simpleAgent:
    def __init__(self, task, model = "gpt-4o-mini", tools = tool_list):
        self.task = task 
        self.model = model
        self.tools = tools
        self.client = establish_client_OpenAI()


        self.user_message = "Calculate the result of the following expression: " + self.task.getCurrentTask() + "."
        self.answer_message = "Now provide your answer in the format Answer: NUMBER where NUMBER is the answer to the question. Only output in this format."
        self.incorrect_message = "Incorrect."
        self.correct_message = "Correct."

        self.messages=[user_message(self.user_message)]

    @retry_with_exponential_backoff
    def get_response_with_tools(self):
        # Generate a response where the model can use tools and add the response to the messages
        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            tools = self.tools,
            tool_choice = "auto"
        )
        self.messages.append(response.choices[0].message)
        return response.choices[0].message

    def get_response_no_tools(self):
        # Generate a response where the model cannot use tools and add the response to the messages
        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )
        self.messages.append(response.choices[0].message)
        return response.choices[0].message
        
    def do_tool_calls(self, message):
        # Do the tool calls and return the response and add the tool calls to the messages
        tool_calls = message.tool_calls
        if tool_calls:
            for tool_call in tool_calls:
                if tool_call.function.name == "calculate":
                    func = getattr(self.task, tool_call.function.name)
                    arguments = json.loads(tool_call.function.arguments)
                    tool_response = func(**arguments)
                    self.messages.append(tool_call_message(tool_call, str(tool_response)))
            return tool_response
        
    def output_and_check_answer(self):
        '''
        Get the model to output a final answer and then check if it is correct, and update the task. 
        Add the final answer to the messages, and tell the model if the answer is correct or incorrect.
        '''
        self.messages.append(user_message(self.answer_message))
        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )
        self.messages.append(response.choices[0].message)
        if self.task.checkAnswer(str(response.choices[0].message.content)[8:]):
            self.messages.append(user_message(self.correct_message))
            self.task.update()
            self.user_message = "Calculate the result of the following expression: " + self.task.getCurrentTask() + "."
            self.messages.append(user_message(self.user_message))
        else:
            self.messages.append(user_message(self.incorrect_message))
        return response.choices[0].message
        
                    



    

### Exercise - Define an agent_loop which uses the agent class to solve the task
```c
Difficulty: 🔴⚪⚪⚪⚪
Importance: 🔵🔵🔵🔵⚪

You should spend up to 5-10 minutes on this exercise.
```

An agent *is* essentially just a loop where a reasoner see what it has done in the past. Try implementing the agent_loop with and without tools, to compare how much better the model does when we give it tools.

In [20]:
task = arithmeticProblem(1500,1091)
agent = simpleAgent(task)

def agent_loop(numLoops = 10):
    for i in range(numLoops):
        if task.checkSolved():
            print("All tasks solved.")
        else:
            response = agent.get_response_with_tools()
            print(response.content)
            tool_response = agent.do_tool_calls(response)
            print(tool_response)
            response = agent.output_and_check_answer()
            print(response.content)

agent_loop()


None
1500 + 1091
2591.0
Answer: 2591
None
1500 - 1091
409.0
Answer: 409
None
1500 * 1091
1636500.0
Answer: 1636500
None
1500 / 1091
1.374885426214482
Answer: 1.374885426214482
None
1500 % 1091
409.0
Answer: 409
None
1500 // 1091
1.0
Answer: 1
All tasks solved.
All tasks solved.
All tasks solved.
All tasks solved.


We can output all the messages from the `ChatHistory` as follows:

In [21]:
for message in agent.messages:
    try: print(str(message.content))
    except: print(message["content"])

Calculate the result of the following expression: 1500 + 1091.
None
2591.0
Now provide your answer in the format Answer: NUMBER where NUMBER is the answer to the question. Only output in this format.
Answer: 2591
Correct.
Calculate the result of the following expression: 1500 - 1091.
None
409.0
Now provide your answer in the format Answer: NUMBER where NUMBER is the answer to the question. Only output in this format.
Answer: 409
Correct.
Calculate the result of the following expression: 1500 * 1091.
None
1636500.0
Now provide your answer in the format Answer: NUMBER where NUMBER is the answer to the question. Only output in this format.
Answer: 1636500
Correct.
Calculate the result of the following expression: 1500 / 1091.
None
1.374885426214482
Now provide your answer in the format Answer: NUMBER where NUMBER is the answer to the question. Only output in this format.
Answer: 1.374885426214482
Correct.
Calculate the result of the following expression: 1500 % 1091.
None
409.0
Now provid

# 3️⃣ Building a More Complex Task

Now that we know how to do function calling, and have a rough understanding of what goes into designing an LM agent, we're going to build a more complicated task. This will enable us to see if we can elicit better capabilities from models.

The task we'll build and elicit behavior for will be the [Wikipedia game](https://en.wikipedia.org/wiki/Wikipedia:Wiki_Game). 

First of all, build a simple function to get a wikipedia page. Probably also mess around with wikipedia api

In [22]:
def get_page(title):
    try: x = wikipedia.page(title,auto_suggest=False,redirect=True)
    except DisambiguationError as e:
        x = wikipedia.page(e.options[0], auto_suggest=False, redirect=True)
    except PageError as e:
        x = wikipedia.page(title, redirect=True, auto_suggest=True)
    return x

### Exercise - Understand the Wikipedia API
```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 15-20 mins on this exercise.
```

Familiarise yourself with the wikipedia API. Make sure you have a good understanding how content is output and structured for different pages, how different links are formatted, how we refer to pages, and why we've set the flags to `auto_suggest=False` and `redirect=True`.

<details><summary> Aside: Wikipedia API content </summary>
<br>
The wikipedia API often outputs content in unintuitive ways. For example, articles that are essentially just a big list become near useless, since the content omits the list (for example, see the wikipedia API content for <a href = "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population">List of countries and dependencies by population</a>). This is fine for our purposes, as these list articles are often banned in wikipedia racing anyway. But things like this is why it's important to determine what is output by the wikipedia API content property — you want to make sure you're testing a large diversity of formattings for articles.</details>

In [23]:
import wikipedia
from wikipedia import WikipediaPage
wikipedia_page = wikipedia.page("GPT-4",auto_suggest=False,redirect=True) # Experiment with different values for auto_suggest and redirect when using the agent. See what happens
#print(wikipedia_page.content)
#print(wikipedia_page.title)
#print(wikipedia_page.html())
#print(wikipedia_page.links)

### Exercise - Get a list of only accessible links from a wikipedia page
```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵⚪⚪⚪

You should spend up to 10-15 mins on this exercise.
```

When writing this function, don't [let the perfect be the enemy of the good enough](https://en.wikipedia.org/wiki/Perfect_is_the_enemy_of_good). If you can get the links in a very effective way, then do that. But remember that Wikipedia is written by a large number of different contributors, often adhering to inconsistent stylings (especially for smaller articles). We just need to get something that works well enough. Put more time into doing this effectively if you want at the end, but as soon as something plausibly works, you should move on.

In [24]:
def get_permitted_links(current_page : WikipediaPage) -> list[str]:
    #Only certain links will be accessible, since the wikipedia api returns a list of ALL possible links (lots of which would not be included in the actual content of the wikipedia, and almost all of which would be banned according to most rules of wiki racing).
    all_links = current_page.links
    content = current_page.content
    permitted_links = []
    for i in all_links:
        if i in content:
            permitted_links.append(i)
    return permitted_links

x = wikipedia.page("Obama", auto_suggest=False, redirect=True)
print(get_permitted_links(x))
print(len(get_permitted_links(x)))
print(len(x.content))

['2003 invasion of Iraq', '2004 Democratic National Convention', '2009 Nobel Peace Prize', '2014 Winter Olympics', '2016 G20 Hangzhou summit', '2020 Democratic presidential primaries', '82nd Airborne Division', 'A Promised Land', 'Abbottabad', 'Abraham Lincoln', 'Academy Award for Best Documentary Feature', 'Affordable Care Act', 'Afghanistan', 'African Americans', 'Air Force One', 'Al Arabiya', 'Alan Keyes', 'American Civil War', 'American Factory', 'American Recovery and Reinvestment Act of 2009', 'Ancestry.com', 'Ann Dunham', 'Anthony Albanese', 'Antiquities Act', 'Anwar al-Awlaki', 'Arab League', 'Arab Spring', 'Arab–Israeli conflict', 'Ares I', 'Ares V', 'Bachelor of Arts', 'Barack', 'Barack Obama 2008 presidential campaign', 'Barack Obama 2012 presidential campaign', 'Barack Obama Presidential Center', 'Barack Obama Sr.', 'Barack and Michelle', 'Ben Bernanke', 'Benjamin Netanyahu', 'Bernie Mac', 'Bill Clinton', 'Biographical Directory of the United States Congress', 'Black Lives 

### Exercise - Wrap links in the content of the wikipedia page
```c
Difficulty: 🔴🔴🔴⚪⚪
Importance: 🔵🔵⚪⚪⚪

You should spend up to 15-20 mins on this exercise.
```

The advice for the `get_permitted_links` function above also applies here. We only need to get something that *works*, and will enable the agent to play the wikipedia game. Come back and spend more time at the end if you want to make this more faithful.

In [25]:
def get_content(page : WikipediaPage) -> str:
    content = page.content
    permitted_links = get_permitted_links(page)
    for word in sorted(permitted_links, key=len, reverse = True):
        content = re.sub(" " + word + " ", " " + f"<link>{word}</link>" + " ", content, flags = re.I)
        content = re.sub(" " + word + ",", " " + f"<link>{word}</link>" + ",", content, flags=re.I)
        content = re.sub(" " + word + ".", " " + f"<link>{word}</link>" + ".", content, flags=re.I)
        content = re.sub("\(" + word + "\)", "(" + f"<link>{word}</link>" + ")", content, flags = re.I)
        content = re.sub(" " + word + "s", " " + f"<link>{word}</link>" + "s", content, flags = re.I)
    return content


# Exercise - Build a class for the Wikipedia game
```c
Difficulty: 🔴🔴🔴🔴⚪
Importance: 🔵🔵🔵🔵⚪

You should spend up to 25-30 mins on this exercise.
```

Implement a class that instantiates the wikipedia game. When the model uses tools it will be making calls to this class, so make sure that the functions return things you're happy for the model to see as a tool response, such as error messages if the tool doesn't work and ensure the formatting of the repsonse will be easily comprehensbile to the LM.

In [26]:
class WikipediaGame:
    def __init__(self, starting_page : str, goal_page : str, rules : list | type[None] = None):
        '''
        Initialises the wikipedia game object

        starting_page is the page the agent starts on.

        goal_page is the page the agent is trying to get to.

        rules is a list of dictionaries specifying any additional rules along with a description of that rule to be fed to the agent.

        Rules that are handled currently:
            - "no country" bans country articles.
            - "no backtrack" bans backtracking.
        '''
        self.page_title_history=[starting_page]
        self.starting_page = get_page(starting_page) # page the game starts on
        self.goal_page = get_page(goal_page) # page the game ends on
        self.rules = rules # any additional rules, (no countries, no articles above a certain length, no backtracking etc. Need to add functionality to deal with these which I haven't done yet)
        self.current_page = get_page(starting_page) # current page the game state is on.

    def get_page_summary(self, page : wikipedia.WikipediaPage):
        '''
        Get summary of a wikipedia page, to the last full stop within the first 500 characters.
        '''
        summary = page.content[0:500]
        return summary[0: summary.rindex(".")+1]

    def move_page(self, **args):
        '''
        Changes the current page of the game. To be used when we want to move from Page A to Page B
        '''
        new_page = args["new_page"]
        if self.is_permitted_link(new_page.lower()):
            self.current_page = get_page(new_page)
            self.page_title_history.append(self.current_page.title)
            return "Moving page to " + self.current_page.title
        elif self.is_permitted_link(new_page.replace("_"," ")):
            self.current_page = get_page(new_page.replace("_"," "))
            self.page_title_history.append(self.current_page.title)
            return "Moving page to " + self.current_page.title
        else:
            return "Couldn't move page to " + new_page

    def get_content(self,**args):
        '''
        Gives the content of the wikipedia page. Make sure that accessible links are wrapped with <link> </link> tags.
        '''
        content = self.current_page.content
        for word in sorted(self.get_permitted_links(), key=len, reverse = True):
            content = re.sub(" " + word + " ", " " + f"<link>{word}</link>" + " ",content, flags = re.I)
            content = re.sub(" " + word + ",", " " + f"<link>{word}</link>" + ",", content, flags=re.I)
            content = re.sub(" " + word + ".", " " + f"<link>{word}</link>" + ".", content, flags=re.I)
            content = re.sub("\(" + word + "\)", "(" + f"<link>{word}</link>" + ")", content, flags = re.I)
            content = re.sub(" " + word + "s", " " + f"<link>{word}</link>" + "s", content, flags = re.I)
        return content

    def get_permitted_links(self):
        #Only certain links will be accessible, since the wikipedia api returns a list of ALL possible links (lots of which would not be included in the actual content of the wikipedia, and almost all of which would be banned according to most rules of wiki racing).
        all_links = self.current_page.links
        content = self.current_page.content
        permitted_links = []
        for i in all_links:
            if i in content:
                permitted_links.append(i)
        return permitted_links

    def is_permitted_link(self, link):
        if link.lower() in (x.lower() for x in self.get_permitted_links()):
            return True
        else:
            return False

    def check_win(self):
        if self.current_page==self.goal_page:
            return True
        else:
            return False

# Exercise - Write a list of tools for this game.
```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵🔵⚪

You should spend up to 10-15 mins on this exercise.
```

Fill in the tools that the agent will need to use to accomplish this game. You should name the tools according to the names in the WikipediaGame class above, so that when we handle the model's tool calling we can access the correct function more easily.

In [81]:
get_content_tool = {
    "type" : "function",
    "function" : {
        "name" : "get_content",
        "description" : "Get all the content for the wikipedia page you are currently on. Anything which corresponds to a link you can select will be wrapped in <link></link> tags.",
        "parameters" : {
            "type" : "object",
            "properties" : {},
            "required" : []
        }
    }
}

move_page_tool = {
    "type" : "function",
    "function" : {
        "name" : "move_page",
        "description" : "Changes your current page to a specified new page which is accessible via a link from the current page. You can only call this function once at a time, as it will take you to a different page.",
        "parameters": {
            "type": "object",
            "properties": {
                "new_page": {
                    "type": "string",
                    "description": "The title of the new page you want to move to. This should be formatted the way the title appears on wikipedia (e.g. to move to the wikipedia page for the United States of America, you should enter \"United States\"). Underscores are not necessary.",
                },
            },
            "required": ["new_page"]
        }
    }
}

WikipediaGameTools = [get_content_tool, move_page_tool]

# Exercise - Build an agent class for the wikipedia racer
```c
🔴🔴🔴🔴⚪
🔵🔵🔵🔵⚪

You should spend up to 30-40 mins on this exercise.
```

Now that you have the `WikipediaGame` class and the tooling set up, build out a `WikipediaRacingAgent` that can access these tools and solve the wikipedia game. Build the agent so that it can be thrown into an agent loop (similar to the one we had for the arithmetic game) without much additional scaffolding. 

There are a few further considerations in this case that we didn't have for the arithmetic game. 

- Since the agent will need to read (potentially very long) wikipedia articles to interact with the game, the context window length becomes relevant. GPT-4o and GPT-4o-mini both have context windows of 128k tokens (which corresponds to ~96k words, but for reference, the wikipedia page for the United States has around 10k words alone and the agent will often need to visit more than 10 articles in one run of the game, not counting its own output, which eventually adds up to be significant). The way we'll solve this for now is by resetting the messages of the agent every time it reaches a new wikipedia page, and providing an updated user and system message so that the agent can locate itself, and then proceed (We'll address different methods for solving this issue later, you can probably already think of some). So be careful to include the current page and goal page for the agent.

- There shouldn't be anything the agent is completely unfamiliar with (AI companies *will* have scraped wikipedia), but it may be easily confused with something else, and models can't always accurately use things in their training data if they only come up once or twice. So you should use the game's get_summary function to provide details of the goal page to the agent in its initial message.

When making calls to the wikipediaGame class, make use of Python's `getattr()` function ([explanation here](https://www.w3schools.com/python/ref_func_getattr.asp)).




In [100]:
class WikipediaRacingAgent:
    def __init__(self, wikigame, model = "gpt-4o-mini", tools = WikipediaGameTools):
        # Initialises the agent
        self.game=wikigame
        self.model = model
        self.client = OpenAI()
        self.tools = tools
        
        self.messages = [self.system_message, self.user_message] # Messages that are currently in the chat history.
        self.all_messages = [self.system_message, self.user_message] # All messages that have been sent in the chat history. We have to erase each time a new page is reached for context window reasons.
        self.response=None

        #print starting messages to watch what the agent is doing
        print("\nSYSTEM: " + self.system_message["content"] + "\n\nUSER: " + self.user_message["content"])

    @property
    def system_message(self):
        return {
            "role" : "system", 
            "content" : "You are a wikipedia-racing AI. Your goal is to reach " + self.game.goal_page.title + " by accessing links from a series of wikipedia pages. Your current page is " + self.game.current_page.title + "."
            }
            
    @property
    def user_message(self):
        return {
            "role" : "user",
            "content" : "You are currently on page: " + self.game.current_page.title + ". Make sure you start by reasoning about what steps you should take to get to the article on " + self.game.goal_page.title + ". When coming up with a strategy, make sure to pay attention to the path you have already taken, and if your current strategy doesn't seem to be working out, try something else. In case you're unsure, " + self.game.goal_page.title + " has the following summary: \n\n[Begin Summary]\n" + self.game.get_page_summary(self.game.goal_page) + "\n[End Summary] " 
            }
    
    @property
    def next_step_message(self):
        return {
            "role" : "user",
            "content" : "What's your next step to get to " + self.game.goal_page.title + "?"
        }
    
    @retry_with_exponential_backoff
    def generate_response_with_tools(self):
        # Generate a response with tools and add it to the messages.
        
        new_response = self.client.chat.completions.create(
            model = self.model,
            messages = self.messages,
            tools = self.tools,
            tool_choice = "auto"
        )

        self.response = new_response.choices[0].message
        self.messages.append(self.response)
        self.all_messages.append(self.response)

        #Print message to watch what the agent is doing
        print("\n" + str(self.response.content))

        return self.response

    def do_tool_calls(self, output):
        # Handle the tool calls and return the response and add the tool calls to the messages
        tool_calls = output.tool_calls
        move_page_count = 0
        if tool_calls:
            for tool_call in tool_calls:
                func = getattr(self.game, tool_call.function.name)
                arguments = json.loads(tool_call.function.arguments)
                new_response = func(**arguments)
                self.messages.append(tool_call_message(tool_call,new_response))
                self.all_messages.append(tool_call_message(tool_call,new_response))
                
                #Print message to watch what the agent is doing
                print("\nTOOL CALL" + tool_call.function.name, arguments)
                print("\nTOOL RESPONSE:" + new_response[:300])

                if tool_call.function.name == "move_page" and "Moving page" in new_response:
                    self.new_state()

                    # TODO: Absolutely NEED to fix this goddamn moving page issue. Really need to think of a non-awful way to solve this problem. (pop things from a list, maybe just learn more about how OpenAI stores these lists)

                    return "new state"
                '''
                elif tool_call.function.name=="move_page" and "Couldn't" in new_response:
                    self.messages.append(user_message("That is not a valid link."))
                    self.all_messages.append(user_message("That is not a valid link."))
                '''
                
            self.messages.append(self.next_step_message)
            self.all_messages.append(self.next_step_message)
            
            #Print message to watch what the agent is doing
            print("\nUSER: What's your next step to get to " + self.game.goal_page.title + "?")
                
                    

    def new_state(self):
        # Begin a new state when a new page is visited. Update user_message and system_message, and reset self.messages (for context window reasons).
        self.messages = [self.system_message,self.user_message]
        self.all_messages.extend([self.system_message,self.user_message])

        #Print message to watch what the agent is doing
        print(("-" * 50) + "\n\nNEW STATE\n\n" + "HISTORY: " + " -> ".join(self.game.page_title_history) + "\n\n" + ("-"*50))
        print("SYSTEM: " + self.system_message["content"] + "\n\nUSER: " + self.user_message["content"])

In [101]:
def agent_loop(agent, game, num_loops = 10):
    for i in range(num_loops):
        if game.check_win():
            print("Success")
            return ""
        response = agent.generate_response_with_tools()
        agent.do_tool_calls(response)

    

In [58]:
game = WikipediaGame("1511 Daléra", "Zalíbená")
agent = WikipediaRacingAgent(game,model="gpt-4o-mini")
agent_loop(agent, game, 10)


SYSTEM: You are a wikipedia-racing AI. Your goal is to reach Zalíbená by accessing links from a series of wikipedia pages. Your current page is 1511 Daléra.

USER: You are currently on page: 1511 Daléra. Make sure you start by reasoning about what steps you should take to get to the article on Zalíbená. When coming up with a strategy, make sure to pay attention to the path you have already taken, and if your current strategy doesn't seem to be working out, try something else. In case you're unsure, Zalíbená has the following summary: 

[Begin Summary]
Zalíbená is a village and administrative part of Podveky in Kutná Hora District in the Central Bohemian Region of the Czech Republic. It has about 20 inhabitants.
[End Summary] 

To reach the article on Zalíbená, I first need to explore the links available on the current page (1511 Daléra) to find any relevant connections to the Czech Republic or the Central Bohemian Region. Since Zalíbená is a village in the Czech Republic, I should loo

In [59]:
for i in agent.messages:
    #try: print(i.content)
    #except: print(i["content"])
    print(i)

{'role': 'system', 'content': 'You are a wikipedia-racing AI. Your goal is to reach Zalíbená by accessing links from a series of wikipedia pages. Your current page is Settlement geography.'}
{'role': 'user', 'content': "You are currently on page: Settlement geography. Make sure you start by reasoning about what steps you should take to get to the article on Zalíbená. When coming up with a strategy, make sure to pay attention to the path you have already taken, and if your current strategy doesn't seem to be working out, try something else. In case you're unsure, Zalíbená has the following summary: \n\n[Begin Summary]\nZalíbená is a village and administrative part of Podveky in Kutná Hora District in the Central Bohemian Region of the Czech Republic. It has about 20 inhabitants.\n[End Summary] "}
ChatCompletionMessage(content='To navigate from the current page "Settlement geography" to the page "Zalíbená," I need to identify relevant links that connect the concepts of geography, settlem

# 4️⃣ Elicitation

Conducting a capability evaluation is necessarily a very tricky task. This is because we're generally trying to show that the model does or dose not have a certain capability (which we're worried about). If the model does have this capability, then we should be able to demonstrate this, and we're done. However, if we fail to demonstrate a capability, that *doesn't* mean the model certainly does not have that capability. Perhaps:

- You prompted the model poorly.

- You stored the long-term history poorly.

- You didn't give the model sufficient tools to accomplish the task.

- There is some additional scaffolding that would drastically increase performance.

It is easy to forget due to its ubiquity today, but it took *3 years* after the release of GPT-2 for people to discover that [Chain-of-thought reasoning improves model performance](https://arxiv.org/abs/2201.11903). A huge potential failure mode for safe AI is a bad actor discovering similar breakthroughs that push model-performance much higher with minimal additional training, so we want ensure we're trying really hard to elicit the best behavior we possibly can, until we feel we've managed to gain [*evidence of absence*](https://en.wikipedia.org/wiki/Evidence_of_absence), not just *absence of evidence*.

Broadly speaking, there are two categories of elicitation, narrow elicitation and general elicitation:

- Narrow elicitation methods are those which will improve model performance on a particular task, or small class of tasks, but won't necessarily impact model performance in a more general way. A good example of narrow elicitation for the wikipedia game might be giving the agent access to the content of arbitrary wikipedia articles. This will improve performance on this task significantly, but wouldn't generalize to a broad array of tasks.

- General elicitation methods are those which will improve model performance on a wide array of possible tasks. A good example of a general elicitation method is chain-of-thought. This tends to improve model performance on a wide array of tasks. These sorts of elicitation methods are the ones we're most interested in, as if bad actors find an improvement to models that is roughly as easy and effective as chain-of-thought prompting, then we would see a very fast increase in risk scenarios overnight.


## Prompting

You should already be aware that prompting can have a large impact on model performance. There are a large number of possible changes to prompts for this task. You should experiment first with more general elicitation methods such as getting the agent to think more deeply, and output plans in different ways. Then you can try a wide array of narrow elicitation methods including:

- Telling the agent how many pages it's visited.

- Telling the agent if it's already visited the page its on (and how many times).

- Schedule different prompts and planning methods for the "zoom out" and "zoom in" sections of the game, since the general strategy for the wikipedia game looks like:

    Specific article (few links) -> General article (many links) -> Specific article (few links)


### Exercise - Try different prompts
```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 20-25 mins on this exercise.
```

Remember that your prompts obviously will have to be robust to: different tasks, different states within those tasks, and different failure-modes the agent could encounter. Mess around with the prompting setup and see if you can significantly improve performance.

In [102]:
class WikipediaRacingAgentPrompting(WikipediaRacingAgent):

    @property
    def system_message(self):
        return {
            "role" : "system", 
            "content" : "You are a wikipedia-racing AI. Your goal is to reach " + self.game.goal_page.title + " by accessing links from a series of wikipedia pages. Your current page is " + self.game.current_page.title + "."
            }
    @property
    def user_message(self):
        return {
            "role" : "user",
            "content" : "WOWOWOWOWOWOW :) You are currently on page: " + self.game.current_page.title + ". Make sure you start by reasoning about what steps you should take to get to the article on " + self.game.goal_page.title + ". When coming up with a strategy, make sure to pay attention to the path you have already taken, and if your current strategy doesn't seem to be working out, try something else. In case you're unsure, " + self.game.goal_page.title + " has the following summary: \n\n[Begin Summary]\n" + self.game.get_page_summary(self.game.goal_page) + "\n[End Summary]"
            }
        


In [103]:
game = WikipediaGame("Aristotle", "Othello")
agent = WikipediaRacingAgentPrompting(game, model="gpt-4o-mini")
agent_loop(agent, game, 5)



SYSTEM: You are a wikipedia-racing AI. Your goal is to reach Othello by accessing links from a series of wikipedia pages. Your current page is Aristotle.

USER: WOWOWOWOWOWOW :) You are currently on page: Aristotle. Make sure you start by reasoning about what steps you should take to get to the article on Othello. When coming up with a strategy, make sure to pay attention to the path you have already taken, and if your current strategy doesn't seem to be working out, try something else. In case you're unsure, Othello has the following summary: 

[Begin Summary]
Othello (; full title: The Tragedy of Othello, the Moor of Venice) is a tragedy written by William Shakespeare around 1603. Set in Venice and Cyprus, the play depicts the Moorish military commander Othello as he is manipulated by his ensign, Iago, into suspecting his wife Desdemona of infidelity. Othello is widely considered one of Shakespeare's greatest works and is usually classified among his major tragedies alongside Macbet

### Exercise - Implement the ReAct framework
```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 10-15 mins on this exercise.
```

Chain-of-thought prompting does confer significant benefits to model performance. But when agents are using tools, there's a better elicitation method. This is called the [ReAct framework](https://arxiv.org/abs/2210.03629). This consists of:

- Getting the model to generate **Re**asoning about its current situation, and what sort of actions it should consider taking.

- Then getting the model to perform an **Act**ion based on its outputted reasoning.

Remember that if you're calling the model without tools, it won't have a description of the tools in its system message, so we'll have to ensure that the tool descriptions are in the `system_message` (this will lead to some redundancy when the model takes an action, but that's alright).

In [104]:
class WikipediaRacingAgentReAct(WikipediaRacingAgentPrompting):

    @property
    def system_message(self):
        #You may or may not want to edit your standard system message
        return {
            "role" : "system",
            "content" : "You are a wikipedia-racing AI. Your goal is to reach " + self.game.goal_page.title + " by accessing links from wikipedia pages. Your current page is " + self.game.current_page.title + ". You have access to " + str(len(self.tools)) + "tools:" + "\n".join([tool["function"]["name"] + ": " + tool["function"]["description"] for tool in self.tools]) + ".\n"
        } #Provided a description of the tools in the system message. When generate is called with tools this is redundant, but when generate is called without tools, this is useful.
    
    @property
    def user_message(self):
        #You may or may not want to edit your standard user message
        return {
            "role" : "user",
            "content" : "You are currently on page: " + self.game.current_page.title + ". Make sure you start by reasoning about what steps you should take to get to the article on " + self.game.goal_page.title + ". When coming up with a strategy, make sure to pay attention to the path you have already taken, and if your current strategy doesn't seem to be working out, try something else. In case you're unsure, " + self.game.goal_page.title + " has the following summary: \n\n[Begin Summary]\n" + self.game.get_page_summary(self.game.goal_page) + "\n[End Summary]."
        }
    
    @retry_with_exponential_backoff
    def generate_response(self, use_tools=True):
        # Generate a response with or without tools and add it to the messages
        new_response = self.client.chat.completions.create(
            model = self.model,
            messages = self.messages,
            tools = self.tools if use_tools else None,
            tool_choice = "auto" if use_tools else None
        )
        return new_response.choices[0].message

    def generate_reason(self):
        # Get the model to reason about the current state of the game and add the response to the messages (you may not want to give it tools for this)
        self.messages.append(user_message("Think carefully about your current situation and what actions you want to take to get closer to" + self.game.goal_page.title + "."))
        response = self.generate_response(use_tools=False)
        self.messages.append(response)
        return response
        
    def generate_action(self):
        # Get the model to generate an action based on the reasoning and add the response to the messages
        self.messages.append(user_message("What action do you want to take?"))
        response = self.generate_response()
        self.messages.append(response)
        return response
    
    def generate_reason_and_action(self):
        # Generate a reason and then an action
        reason = self.generate_reason()
        print(reason.content)
        action = self.generate_action()
        print(action.content)
        self.do_tool_calls(action)
        return action


You may have to rewrite your `agent_loop`.

In [106]:
def agent_loop_ReAct(game, agent, numLoops = 10):
    for i in range(numLoops):
        if game.check_win():
            print("Success")
            return ""
        agent.generate_reason_and_action()

In [74]:
game = WikipediaGame("Aristotle", "Othello")
agent = WikipediaRacingAgentReAct(game, model="gpt-4o-mini", tools = WikipediaGameTools)
agent_loop_ReAct(game, agent)



SYSTEM: You are a wikipedia-racing AI. Your goal is to reach Othello by accessing links from wikipedia pages. Your current page is Aristotle. You have access to 2tools:get_content: Get all the content for the wikipedia page you are currently on. Anything which corresponds to a link you can select will be wrapped in <link></link> tags.
move_page: Changes your current page to a specified new page which is accessible via a link from the current page. You can only call this function once at a time, as it will take you to a different page..


USER: You are currently on page: Aristotle. Make sure you start by reasoning about what steps you should take to get to the article on Othello. When coming up with a strategy, make sure to pay attention to the path you have already taken, and if your current strategy doesn't seem to be working out, try something else. In case you're unsure, Othello has the following summary: 

[Begin Summary]
Othello (; full title: The Tragedy of Othello, the Moor of 

In [75]:
for i in agent.messages:
    print(i)

{'role': 'system', 'content': 'You are a wikipedia-racing AI. Your goal is to reach Othello by accessing links from wikipedia pages. Your current page is Aristotle. You have access to 2tools:get_content: Get all the content for the wikipedia page you are currently on. Anything which corresponds to a link you can select will be wrapped in <link></link> tags.\nmove_page: Changes your current page to a specified new page which is accessible via a link from the current page. You can only call this function once at a time, as it will take you to a different page..\n'}
{'role': 'user', 'content': "You are currently on page: Aristotle. Make sure you start by reasoning about what steps you should take to get to the article on Othello. When coming up with a strategy, make sure to pay attention to the path you have already taken, and if your current strategy doesn't seem to be working out, try something else. In case you're unsure, Othello has the following summary: \n\n[Begin Summary]\nOthello 

## Scaffolding

We can improve the scaffolding by:
- Telling the agent its history.
- Giving the agent a "reflexion tool" (link reflexion paper)
- The prompts are also technically a part of the "scaffolding"
- We can tell the agent if it's already visited a page.

### Exercise - Implement an agent history property

You may notice that the agent frequently gets stuck in loops. Since we're already storing the page title history in the game class, we should probably provide this information to the agent and see if it improves this looping behavior. Implement this below.

In [118]:
class WikipediaRacingAgentHistory(WikipediaRacingAgentReAct):
    @property
    def system_message(self):
        return {
            "role" : "system",
            "content" : "You are a wikipedia-racing AI. Your goal is to reach " + self.game.goal_page.title + " by accessing links from a series of wikipedia pages. Your current page is " + self.game.current_page.title + ". You have access to " + str(len(self.tools)) + "tools:\n" + "\n".join([tool["function"]["name"] + ": " + tool["function"]["description"] for tool in self.tools]) + ".\n"
            }
    
    @property
    def user_message(self):
        return {
            "role" : "user",
            "content" : "You are currently on page: " + self.game.current_page.title + ". Make sure you start by reasoning about what steps you should take to get to the article on " + self.game.goal_page.title + ". When coming up with a strategy, make sure to pay attention to the path you have already taken, and if your current strategy doesn't seem to be working out, try something else. In case you're unsure, " + self.game.goal_page.title + " has the following summary: \n\n[Begin Summary]\n" + self.game.get_page_summary(self.game.goal_page) + "\n[End Summary]\n The pages you've visited so far has been: " + " -> ".join(self.game.page_title_history)
        }

### Exercise - Implement a reflexion tool

This paper (link reflexion paper) finds better performance by LLMs on tasks when they can perform "lookahead" on their plans. We will imitate this by allowing the agent to suggest candidate paths, and informing it where these paths err (if they do). You'll need to add this tool to the list of tools.

In [121]:
class WikipediaGameTestPath(WikipediaGame):
    def __init__(self, starting_page : str, goal_page : str, rules : list | type[None] = None):
        super().__init__(starting_page, goal_page, rules)

    def get_permitted_links_any_page(self, title):
        #Only certain links will be accessible, since the wikipedia api returns a list of ALL possible links (lots of which would not be included in the actual content of the wikipedia, and almost all of which would be banned according to most rules of wiki racing).
        all_links = get_page(title).links
        content = get_page(title).content
        permitted_links = []
        for i in all_links:
            if i.lower() in content.lower():
                permitted_links.append(i)
        return permitted_links
    def test_path(self, path : str) -> str:
        '''
        Reflexion test_path function, takes a dict like {"path": "Barack Obama -> Indonesia -> India"} and returns True if the path works, for a path like "Barack Obama -> Indonesia -> Pink Floyd" it would return "This path works until Pink floyd, which is not accessible from Indonesia".
        '''
        print("\n\n\n\n\n\n\n\n\n Test path \n\n\n\n\n\n\n\n\n")
        path = path.split("->")
        if path[0].strip() != self.current_page.title:
            return "ERROR: The title of the start page of this path is not the title of your current page."
        for i in range(len(path)-1):
            if path[i+1].strip().lower() in [link.lower() for link in self.get_permitted_links_any_page(path[i])]:
                continue
            else:
                return "This path works until " + path[i+1].strip() + ", which is not accessible from " +path[i].strip()
        return "This path completely works."


Now write a description, and add it to the list of `WikipediaGameTools`.

In [122]:
test_path_tool = {
    "type" : "function",
    "function" : {
        "name" : "test_path",
        "description" : "Accepts a test path string in the form \"current_page -> page1 -> page2 -> ... -> pageN\" and if the path does not work, then it returns where the path goes wrong, if the path does work it returns \"success.\" Be careful that path titles can be sensitive to plurals or rephrasings. This tool is especially useful to check longer plans.",
        "parameters" : {
            "type" : "object",
            "properties": {
                "path" : {
                    "type" : "string",
                    "description" : "The path you want to test, formatted as \" current_page -> page1 -> page2 -> ... -> pageN\"."
                },
            },
            "required" : ["path"]
        }
    }
}

WikipediaGameTools = [get_content_tool, move_page_tool, test_path_tool]

In [123]:
game = WikipediaGameTestPath("William Pitt the Younger", "Central Vietnam")
agent = WikipediaRacingAgentHistory(game, model="gpt-4o-mini", tools = WikipediaGameTools)
agent_loop_ReAct(game,agent, 40)


SYSTEM: You are a wikipedia-racing AI. Your goal is to reach Central Vietnam by accessing links from a series of wikipedia pages. Your current page is William Pitt the Younger. You have access to 3tools:
get_content: Get all the content for the wikipedia page you are currently on. Anything which corresponds to a link you can select will be wrapped in <link></link> tags.
move_page: Changes your current page to a specified new page which is accessible via a link from the current page. You can only call this function once at a time, as it will take you to a different page.
test_path: Accepts a test path string in the form "current_page -> page1 -> page2 -> ... -> pageN" and if the path does not work, then it returns where the path goes wrong, if the path does work it returns "success." Be careful that path titles can be sensitive to plurals or rephrasings. This tool is especially useful to check longer plans..
 The first tool you try to use is the test_path tool. You always test at least

''

## Tool use (might be worth experimenting with this somewhat anyway)

We could also give the agent additional tools, but if you give the agent too many tools (especially with poor descriptions), then performance can often suffer. This happens most prominently when using more than 5-10 tools.

### Exercise - Implement a page summary tool
```c
Difficulty:🔴🔴⚪⚪⚪
Importance:🔵🔵⚪⚪⚪

You should spend up to 10-15 mins on this exercise.
```

Implement a tool that allows an agent to get a summary of a page before moving to it. This imitates wikipedia's native 'hover summary' tool.


In [125]:
get_page_summary = {
    "type" : "function",
    "function" : {
        "name" : "get_page_summary",
        "description" : "Get the summary of a wikipedia page you are considering moving to, to the last full stop within the first 500 characters. The page needs to be accessible via a link from the current page. Anything which corresponds to a link you can select will be wrapped in <link></link> tags",
        "parameters" : {
            "type" : "object",
            "properties" : {
                "page" : {
                    "type" : "object",
                    "description" : "The wikipedia page you want to get the summary of."
                }
            },
            "required" : ["page"]
        }
    }
}

class WikipediaGamePageSummary(WikipediaGameTestPath):
    def __init__(self, starting_page : str, goal_page : str, rules : list | type[None] = None):
        super().__init__(starting_page, goal_page, rules)

        
    def get_page_summary(self, page : WikipediaPage) -> str:
        if is_permitted_link(self, page):
            summary = page.content[0:500]
            return summary[0: summary.rindex(".")+1]
        else:
            return "This page is not accessible from the current page, so a summary cannot be returned."

WikipediaGameTools.append(get_page_summary)

### Exercise - Implement an arbitrary page summary/content tool
```c
Difficulty:🔴🔴⚪⚪⚪
Importance:🔵🔵⚪⚪⚪

You should spend up to 10-15 mins on this exercise.
```


In [ ]:
get_page_content = {
    "type" : "function",
    "function" : {
        "name" : "get_page_content",
        "description" : "Get the content of a wikipedia page you are considering moving to. Anything which corresponds to a link you can select will be wrapped in <link></link> tags.",
        "parameters" : {
            "type" : "object",
            "properties" : {
                "page" : {
                    "type" : "object",
                    "description" : "The wikipedia page you want to get the content of."
                }
            },
            "required" : ["page"]
        }
    }
}

class WikipediaGamePageContent(WikipediaGamePageSummary):
    def __init__(self, starting_page : str, goal_page : str, rules : list | type[None] = None):
        super().__init__(starting_page, goal_page, rules)
    def get_page_content(self, arguments : dict) -> str:
        page = arguments["page"]
        content = page.content
        return content

### Exercise - Implement a ctrl-F tool

Still need to do this. Probably will though. Not super urgent. Might add more elicitation stuff later if I think of any that seem cool

# 5️⃣ Bonus

### Exercise - Implement additional rules

Test agent performance on these tasks:
- Task 1

- Task 2

- Task 3

- Task 4

- Task 5

- Task 6

- Task 7

- Task 8

- Task 9

- Task 10

See what combination of tools appears to work best.

### Exercise - Rearrange so that each page is broken up by sections

In [70]:
x = get_page("Aristotle")
print(dir(x))
print(x.section("Metaphysics/Substance"))

['_WikipediaPage__continued_query', '_WikipediaPage__load', '_WikipediaPage__title_query_param', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'categories', 'content', 'coordinates', 'html', 'images', 'links', 'original_title', 'pageid', 'parent_id', 'references', 'revision_id', 'section', 'sections', 'summary', 'title', 'url']
None
